In [1]:
import asyncio
import datetime
import os
import warnings

from dotenv import load_dotenv, find_dotenv
from langsmith import Client

_ = load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")

unique_id = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Trip Planner Tracing | Run - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

client = Client()

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.llms import GooglePalm
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool, load_tools, initialize_agent
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

In [3]:
system_prompt_template = """
    The following is a conversation between a human and a smart AI itinerary planner. Answer only questions based on trip planning.
    
    The AI is an expert at planning trips. As a smart itinerary planner with extensive knowledge of places around the world, 
    your task is to determine the user's travel destinations and any specific interests or preferences from their message by keeping in mind weather, 
    geographical proximity, optimized routes to minimize travel time.
    
    The plan should contain at least 4 day-to-day activities along with timings to visit based on the weather and estimted cost.
    
    If the AI does not know the answer to a question, it truthfully says it does not know.

    Current conversation:
    {history}
    Human: {input}
    AI:"""


def load_user_template(query):
    template = f"{query}. Consider weather and budget, and include estimated cost for each activity. Display the ititenary in the following format: 'Trip to <country/city/place>\n\nDay 1: <weather>\n<time-to-visit><things to do> or <places to visit> : \\<estimated cost>\n<time-to-visit><things to do> or <places to visit> : \\<estimated cost>\n<time-to-visit><things to do> or <places to visit> : \\<estimated cost>\n<time-to-visit><things to do> or <places to visit> : <estimated cost>\n\nDay 2: <weather>\n<time-to-visit><things to do> or <places to visit> : <estimated cost>\n<time-to-visit><things to do> or <places to visit> : <estimated cost>\n<time-to-visit><things to do> or <places to visit> : <estimated cost>\n<time-to-visit><things to do> or <places to visit> : <estimated cost>\n\nTotal Estimated Cost: <total estimated cost>\n\nEnjoy your Trip!!!' Don't include extra information."
    return template

In [4]:
PROMPT = PromptTemplate(
    input_variables=["history", "input"], 
    template=system_prompt_template
)

googlepalm_llm = GooglePalm(
    temperature=0.1, 
    top_k=3
)
conversationchain = ConversationChain(
    llm=googlepalm_llm, 
    prompt=PROMPT
)

llm_tool = Tool(
    name="Language Model",
    func=conversationchain.run,
    description="use this tool for generating itinerary plan from the human input by considering weather",
)
tools = load_tools(
    ["openweathermap-api"], 
    llm=googlepalm_llm
)
tools.append(llm_tool)

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=googlepalm_llm,
    verbose=True,
    memory=memory,
    max_iterations=3,
    early_stopping_method="generate",
    handle_parsing_errors=True,
)

In [5]:
inputs = [
    "Plan a 3 day trip to NYC from July 22nd to July 24th",
    "Generate an itinerary to Boston for 2 days with $300 budget",
    "Plan a trip to Seattle with $500 budget",
    "4 day itinerary to Miami with $600 budget from Aug 15th to Aug 18th"
]
results = []


async def arun(agent, input_example, chat_history):
    try:
        return await conversational_agent(
        {
            "chat_history": chat_history, 
            "input": input_example
        }
    )
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


for input_example in inputs:
    chat_history = conversationchain.memory
    input_example = load_user_template(input_example)
    results.append(arun(conversational_agent, input_example, chat_history))
results = await asyncio.gather(*results)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: OpenWeatherMap
Action Input: New York City, US
Observation: In New York City, US, the current weather is as follows:
Detailed status: haze
Wind speed: 4.12 m/s, direction: 50°
Humidity: 66%
Temperature: 
  - Current: 25.84°C
  - High: 28.34°C
  - Low: 23.75°C
  - Feels like: 26.2°C
Rain: {}
Heat index: None
Cloud cover: 100%
Thought:Do I need to use a tool? No
AI: **Trip to New York City**

**Day 1:**

* **Weather:** Partly cloudy, with a high of 28°C and a low of 23°C.
* **10:00 AM - 12:00 PM:** Visit the Statue of Liberty and Ellis Island. Tickets cost $28 for adults and $19 for children.
* **12:00 PM - 2:00 PM:** Have lunch at a local restaurant. The average cost of a meal is around $20 per person.
* **2:00 PM - 4:00 PM:** Visit the Metropolitan Museum of Art. Admission is free.
* **4:00 PM - 6:00 PM:** Take a walk through Central Park. The park is free to visit.

**Day 2:**

* **Weather:** Sunny, 

In [6]:
wait_for_all_tracers()